<a href="https://colab.research.google.com/github/Nida0709/lab/blob/main/FrictionCoefficient_Outcome_Comparing_tool_(Tribology).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#fileListから情報取得
def sprit_fileList(data_Path):

  import os

  #Body
  buf = os.path.basename(data_Path)
  fileName = buf

  countN = buf.find("_")
  Method = buf[ : countN]
  buf = buf[countN+1 : ]

  countN = buf.find("_")
  Material = buf[ : countN]
  buf = buf[countN+1 : ]

  countN = buf.find("_")
  Weight = buf[ : countN]
  buf = buf[countN+1 : ]

  countN = buf.find("_")
  SlidingSpeed = buf[ : countN]
  buf = buf[countN+1 : ]

  countN = buf.find(".")
  number = int(buf[ : countN])

  return fileName, Method, Material, Weight, SlidingSpeed, number

In [ ]:
from operator import index
#DB→Excel出力
def export_DB(dataPath, DB, header):
  #Excel操作系ライブラリーのインポート
  import pandas
  df = pandas.DataFrame(DB)
  df.to_excel("/content/drive/MyDrive/研究/比較データ/" + dataPath, index=False, header=header)

  frag = "True"
  return frag

fileList[]:参照データ全てのPath
method_name:決定した比較モードの名前格納
count_fL:参照データの通し番号
data_Path=fileList[count_fL]
DB[[]]:参照データ(Excel),dataframe形式
SlidingTime:摺動時間
FrictionCoefficient:摩擦係数
fileName:参照データのファイル名
Method:参照データの平滑モード
Material:参照データの素材
Weight:参照データの試験荷重
SlidingSpeed:参照データの試験線速度
number:参照データの実験通し番号
slope:参照データの隣り合った2点間の傾き
slope_Outcome:出力用参照データの傾き,ave_slopeのデータ利用
delta:どれだけのデータを平均するか
ave_slope:slope200点(20sec)の平均
new_data_Path:作成するデータのPath
frag_process:実行結果格納

In [ ]:
def function1(fileList, method_name, delta):    #delta=平均する微分の範囲
  import pandas


  #Body
  for count_fL in range(len(fileList)):     #count_fL = count_fileList
    print(str(count_fL+1) + "/" + str(len(fileList)))

    #DB取得
    data_Path = fileList[count_fL]
    DB = pandas.read_excel(data_Path, sheet_name = "Sheet1")

    #DB→計算用一次元配列に置き換え
    SlidingTime = []
    FrictionCoefficient = []
    for i in range(len(DB)-19):
      SlidingTime.append(DB.values[i+19][0])
      FrictionCoefficient.append(DB.values[i+19][2])
    
    #fileList→Param取得
    fileName, Method, Material, Weight, SlidingSpeed, number = sprit_fileList(data_Path)

    #DB→differential
    slope = []
    slope_Outcome = []
    for n in range(len(SlidingTime)-1):
      slope.append((FrictionCoefficient[n + 1] - FrictionCoefficient[n]) / (SlidingTime[n + 1] - SlidingTime[n]))
      if n%delta == delta - 1:
        for i in range(delta):
          ave_slope = (slope[n-(delta-1) + i] / delta)
        slope_Outcome.append([SlidingTime[n-(delta-1)], ave_slope])
        ave_slope = 0

    #DB→Excel出力
    new_fileName = method_name + "/" + fileName
    header = ["time","slope"]
    frag_process = export_DB(new_fileName, slope_Outcome, header)
    if frag_process == "True":
      print("seved file.")
    else:
      print("error02\n出力失敗しました。")

fileList[]:参照データ全てのPath
method_name:決定した比較モードの名前格納
count_fL:参照データの通し番号
data_Path=fileList[count_fL]

In [ ]:
def function2(fileList, method_name, delta):      #delta=平均する微分の範囲
  import pandas


  #Body
  Range_Outcome = []
  for count_fL in range(len(fileList)):     #count_fL = count_fileList
    print(str(count_fL+1) + "/" + str(len(fileList)))

    #DB取得
    data_Path = fileList[count_fL]
    DB = pandas.read_excel(data_Path, sheet_name = "Sheet1")

    #ここにExcelの入力を作成＆countloopを利用
    #fileList→Param取得
    fileName, Method, Material, Weight, SlidingSpeed, number = sprit_fileList(data_Path)

    #DB→計算用一次元配列に置き換え
    SlidingTime = []
    FrictionCoefficient = []
    for i in range(len(DB)-19):
      SlidingTime.append(DB.values[i+19][0])
      FrictionCoefficient.append(DB.values[i+19][2])

    #DB→Range[delta]→average_Range[delta]
    max = 0
    min = 1000
    max_range = 0
    average_range = 0
    for n in range(len(SlidingTime)):
      if FrictionCoefficient[n] > max:
        max = FrictionCoefficient[n]
      if FrictionCoefficient[n] < min:
        min = FrictionCoefficient[n]
      #1区間の計測が終了した場合
      if n%delta == delta-1:
        if (max-min) > max_range:
          max_range = (max-min)
        average_range = average_range + (max - min) *delta / len(SlidingTime)
        max = 0
        min = 1000
    Range_Outcome.append([Method, Material, Weight, SlidingSpeed, number, max_range, average_range])

  #DB→Excel出力
  new_fileName = method_name + ".xlsx"
  header = ["Method", "Material", "Weight", "SlidingSpeed", "number", "max_Range", "average_Range"]
  frag_process = export_DB(new_fileName, Range_Outcome, header)
  if frag_process == "True":
    print("seved file.")
  else:
    print("error02\n出力失敗しました。")
          

frag_method:比較モードのデータ
method_index[]:比較モードの名前格納
frag_confirm:実行最終確認
method_name：決定した比較モードの名前格納
target_Path:参照データ保存先Path
fileList[]:参照データ全てのPath

In [ ]:
from numpy import sqrt
def function3(fileList, method_name, delta):      #delta=平均する微分の範囲
  import pandas


  #Body
  SD_Outcome = []
  for count_fL in range(len(fileList)):     #count_fL = count_fileList
    print(str(count_fL+1) + "/" + str(len(fileList)))

    #DB取得
    data_Path = fileList[count_fL]
    DB = pandas.read_excel(data_Path, sheet_name = "Sheet1")

    #ここにExcelの入力を作成＆countloopを利用
    #fileList→Param取得
    fileName, Method, Material, Weight, SlidingSpeed, number = sprit_fileList(data_Path)

    #DB→計算用一次元配列に置き換え
    SlidingTime = []
    FrictionCoefficient = []
    for i in range(len(DB)-19):
      SlidingTime.append(DB.values[i+19][0])
      FrictionCoefficient.append(DB.values[i+19][2])

    #DB→Standard Deviation[delta]→average_Standard Deviation[delta]
    #区間平均をとる
    average_FC = 0
    average_SD = 0
    for n in range(len(SlidingTime)):
      average_FC = FrictionCoefficient[n] / delta
      #1区間の計測が終了した場合
      if n%delta == delta-1:
        for i in range(n-19, n):
          #分散の計算
          Distributed = (FrictionCoefficient[i] - average_FC) * (FrictionCoefficient[i] - average_FC) / delta
        #標準偏差の平均
        average_SD = sqrt(Distributed) *delta / len(SlidingTime)
        average_FC = 0

    SD_Outcome.append([Method, Material, Weight, SlidingSpeed, number, average_SD])

  #DB→Excel出力
  new_fileName = method_name + ".xlsx"
  header = ["Method", "Material", "Weight", "SlidingSpeed", "number", "average_Standard Deviation"]
  frag_process = export_DB(new_fileName, SD_Outcome, header)
  if frag_process == "True":
    print("seved file.")
  else:
    print("error02\n出力失敗しました。")

In [ ]:
#摩擦係数の評価
#必要ライブラリー
import google.colab
from google.colab import files
import sys
import pandas
import glob

method_index = ["20秒間の平均傾き","20秒間の平均レンジ","20秒間の平均標準偏差","xx秒間の平均レンジ","xx秒間の平均標準偏差"]

#Body
#処理種別決定&処理内容確認
print("計算手法を入力してください。")
for i in range(len(method_index)):
  print(str(i) + ":" + method_index[i])
frag_method = int(input(">>"))
#xx秒系の時間範囲入力
if (frag_method == 3) or (frag_method == 4):
  rangeTime = int(input("時間範囲を入力してください。[sec]\n>>"))
  method_index[frag_method] = str(rangeTime) + method_index[frag_method][2: ]      #methodの変更
for i in range(5):
  frag_confirm = input(str(method_index[frag_method]) + "を実行します。\n処理を続行するならばyes,キャンセルするならばnoを入力してください。\n>>")
  if frag_confirm == "yes":
    print("処理開始を確認")
    break
  elif frag_confirm == "no":
    print("処理失敗を確認")
    sys.exit()
  else:
    print("error01")
    print("再度入力してください。")
    if i == 4:
      print("上限を超えました。")
      sys.exit()
method_name = method_index[frag_method]

#fileList取得
target_Path = "/content/drive/MyDrive/研究/実験データ/*.xlsx"
fileList = glob.glob(target_Path)
if len(fileList) == 0:
  print("error03\nデータがありません。システムを終了します。")
  sys.exit()


#20秒間の平均傾き
if frag_method == 0:
  delta = 200
  function1(fileList, method_name, delta)

#20秒間の平均レンジ
elif frag_method ==1:
  delta = 200
  function2(fileList, method_name, delta)

#20秒間の平均標準偏差
elif frag_method == 2:
  delta = 200
  function3(fileList, method_name, delta)

#xx秒間の平均レンジ
elif frag_method == 3:
  delta = rangeTime *10
  function2(fileList, method_name, delta)

#xx秒間の平均標準偏差
elif frag_method == 4:
  delta = rangeTime *10
  function3(fileList, method_name, delta)

else:
  print("error04\nモードがありません。システムを終了します。")
  sys.exit()

print("All Process is Done\nstop Running\nSee you my Boss")

比較フォルダーにファイルがある場合削除するか否か問う
自動でフォルダー作成する